In [ ]:
# # Google Drive 마운트 (선택 사항: 데이터 및 모델 저장을 위해 필요할 수 있습니다)
# from google.colab import drive
# drive.mount('/content/drive')

# Colab 기본 작업 디렉토리 설정 (필요시)
!pip install pandas torch transformers peft trl datasets scikit-learn tqdm accelerate bitsandbytes


In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import torch
import transformers,peft,trl, sklearn, accelerate, bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, set_seed, TrainingArguments
from peft import PeftModel, LoraConfig, get_peft_model, prepare_model_for_kbit_training
import re
from tqdm import tqdm
import argparse
import os
from typing import List, Tuple, Dict, Any, Union
import time
import random
import numpy as np
import itertools
from datasets import Dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from sklearn.model_selection import train_test_split
import gc
import shutil


print("--- Installed Library Versions ---")
print(f"pandas version: {pd.__version__}")
print(f"torch version: {torch.__version__}")
print(f"transformers version: {transformers.__version__}")
print(f"peft version: {peft.__version__}")
print(f"trl version: {trl.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
try:
    print(f"accelerate version: {accelerate.__version__}")
except AttributeError:
    print("accelerate version: Not directly available or not installed.")
try:
    print(f"bitsandbytes version: {bitsandbytes.__version__}")
except AttributeError:
    print("bitsandbytes version: Not directly available or not installed.")
print("--------------------------------")

--- Installed Library Versions ---
pandas version: 2.2.2
torch version: 2.6.0+cu124
transformers version: 4.52.4
peft version: 0.15.2
trl version: 0.19.0
scikit-learn version: 1.6.1
accelerate version: 1.8.1
bitsandbytes version: 0.46.0
--------------------------------


In [ ]:
# 모델 설정
MODEL_NAME = "Qwen/Qwen3-8B"  # 베이스 모델
HUGGINGFACE_REPO = "z0104241/DACON_sentence_order"
ADAPTER_SUBFOLDER = "qwen3_model"
MAX_SEQ_LENGTH = 2048

# 양자화 설정
USE_4BIT = True
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"
BNB_4BIT_QUANT_TYPE = "nf4"
BNB_4BIT_USE_DOUBLE_QUANT = False

# LoRA 설정
LORA_R = 64
LORA_ALPHA = 128
LORA_DROPOUT = 0.05
LORA_TARGETS = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# === A5000(24GB), 증강데이터 기준 파라미터 ===
LEARNING_RATE = 2e-4    # 더 큰 데이터, 배치크기 증가에 맞춰 살짝 상향
BATCH_SIZE = 8         # 단일 GPU에서 4bit, LoRA시 24GB 충분
GRAD_ACCUMULATION = 8   # Effective batch size 32 (4x8)
MAX_STEPS = 800        # (데이터 28k / 32 = 약 900step/epoch) → 2~3epoch 정도 커버
WARMUP_STEPS = 200
SAVE_STEPS = 10000        # 더 자주 저장

# 추론 설정 등 기타 동일
TRAIN_FILE = "train.csv"
TRAIN_AUG_FILE = "train_augmented_kor.csv"
TEST_FILE = "test.csv"
OUTPUT_DIR = "qwen3_model"
PREDICTIONS_FILE = "predictions.csv"
CACHE_DIR = "./model_cache"

# 필요한 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs("outputs", exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

TEMPERATURE = 0.1
TOP_P = 1.0
# TOP_K = 20
MAX_NEW_TOKENS = 150
INFERENCE_BATCH_SIZE = 8  # 배치 처리용
CHECKPOINT_INTERVAL = 10000  # 체크포인트 저장 간격

# 프롬프트 설정
FEWSHOT_EXAMPLE = """예시:
문장들:
0: 119에 신고했다.
1: 아파트에서 화재가 발생했다.
2: 소방차가 현장에 도착했다.
3: 불이 완전히 진화되었다.
답: 1,0,2,3
"""

PROMPT_TEMPLATE = (
    "다음은 문장 순서 배열의 예시입니다. 문맥을 파악하여 가장 자연스러운 순서를 찾으세요.\n\n"
    + FEWSHOT_EXAMPLE +
    "이제 다음 문장들을 배열하세요:\n\n"
    "문장들:\n"
    "0: {sentence_0}\n"
    "1: {sentence_1}\n"
    "2: {sentence_2}\n"
    "3: {sentence_3}\n"
    "답: <|im_start|>assistant"
)

# 2. 데이터 증강 (augment_llama3.py 내용)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
set_seed(SEED)

AUGMENT_MODEL_NAME = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

def make_prompt(sentence):
    return (
        "llm 학습용 데이터를 증강할거야. 아래 문장을 의미를 바꾸지 않고 다양한 표현으로 페러프레이징 해줘. **절대로 다른 언어는 사용하지말고 한국어만 사용해.**\n"
        "예시:\n"
        "문장: 인공지능은 미래 사회를 변화시킬 중요한 기술이다.\n"
        "출력: 미래 사회를 바꿀 핵심 기술 중 하나가 바로 인공지능이다.\n"
        "문장: 데이터 분석은 기업 의사결정에 큰 영향을 준다.\n"
        "출력: 기업이 의사결정을 내릴 때 데이터 분석이 중요한 역할을 한다.\n"
        f"문장: {sentence}\n"
        "출력:"
    )

def hamming_distance(a, b):
    return sum(x != y for x, y in zip(a, b))

def cleanup():
    print("모델, 캐시, VRAM 비우는 중...")
    global augment_model, augment_tokenizer # 전역 변수로 선언된 augment_model, augment_tokenizer 사용
    try:
        del augment_model
    except Exception:
        pass
    try:
        del augment_tokenizer
    except Exception:
        pass

    try:
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    except Exception:
        pass

    gc.collect()

    hf_cache_dirs = [
        os.path.expanduser("~/.cache/huggingface"),
        os.path.expanduser("~/.cache/torch/transformers"),
        os.getenv("TRANSFORMERS_CACHE", ""),
        os.getenv("HF_HOME", ""),
    ]
    for d in hf_cache_dirs:
        if d and os.path.exists(d):
            print(f"캐시 폴더 삭제: {d}")
            shutil.rmtree(d, ignore_errors=True)

    print("메모리, 캐시 정리 완료")

def run_augmentation():
    print("데이터 증강 시작...")
    global augment_tokenizer, augment_model, generator # 전역 변수로 선언
    augment_tokenizer = AutoTokenizer.from_pretrained(AUGMENT_MODEL_NAME)
    augment_model = AutoModelForCausalLM.from_pretrained(
        AUGMENT_MODEL_NAME,
        torch_dtype="auto",
        device_map={"": "cuda:0"}
    )

    generator = pipeline(
        "text-generation",
        model=augment_model,
        tokenizer=augment_tokenizer,
        max_new_tokens=50,
        temperature=1.1,
        do_sample=True,
        top_p=0.92,
        repetition_penalty=1.15
    )

    BATCH_SIZE_AUG = 3  # VRAM에 따라 조정

    # === train.csv 읽기 ===
    df = pd.read_csv(TRAIN_FILE) # TRAIN_FILE 사용

    # sentence 컬럼명 추출
    cols = [c for c in df.columns if c.startswith("sentence_")]
    ans_cols = [c for c in df.columns if c.startswith("answer_")]

    # ----------------------
    # 1. 파라프레이즈 데이터 생성 (증강행)
    # ----------------------
    to_paraphrase = []
    for idx, row in df.iterrows():
        for col in cols:
            to_paraphrase.append((idx, col, row[col]))

    prompts = [make_prompt(x[2]) for x in to_paraphrase]
    paraphrased_results = []
    total_batches = (len(prompts) + BATCH_SIZE_AUG - 1) // BATCH_SIZE_AUG
    for i in tqdm(range(0, len(prompts), BATCH_SIZE_AUG), total=total_batches):
        batch_prompts = prompts[i:i+BATCH_SIZE_AUG]
        outputs = generator(batch_prompts)
        for j, out in enumerate(outputs):
            text = out[0]["generated_text"]
            text = text.replace(batch_prompts[j], '').strip()
            paraphrased = text.split('\n')[0]
            if "출력:" in paraphrased:
                paraphrased = paraphrased.split("출력:")[-1].strip()
            paraphrased = paraphrased.lstrip('-').strip()
            paraphrased_results.append(paraphrased)

    # ----------------------
    # 2. 파라프레이즈된 증강 행 만들기 (단, 나중에 순서만 섞어서 쓸 것)
    # ----------------------
    result_idx = 0
    paraphrased_aug_rows = []
    for idx, row in df.iterrows():
        new_row = row.copy()
        new_row["ID"] = str(row["ID"]) + "_aug"
        for col in cols:
            new_row[col] = paraphrased_results[result_idx]
            result_idx += 1
        paraphrased_aug_rows.append(new_row)
    aug_df = pd.DataFrame(paraphrased_aug_rows)

    # ----------------------
    # 3. 순서 섞기 (파라프 행만)
    # ----------------------
    max_aug = 1  # 한 행 당 몇 개 생성할지
    reordered_rows = []
    for _, row in aug_df.iterrows():
        orig_perm = tuple([0, 1, 2, 3])
        all_perms = list(itertools.permutations(range(4)))
        hard_perms = [p for p in all_perms if 1 <= hamming_distance(orig_perm, p) <= 2]
        if not hard_perms:
            continue
        selected = random.sample(hard_perms, min(max_aug, len(hard_perms)))
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        for idx2, perm in enumerate(selected, 1):
            aug_sentences = [sentences[i] for i in perm]
            aug_row = row.copy()
            for i in range(4):
                aug_row[f'sentence_{i}'] = aug_sentences[i]
                aug_row[f'answer_{i}'] = perm[i]
            aug_row['ID'] = f"{row['ID']}_{idx2}"
            reordered_rows.append(aug_row.copy())
    reordered_df = pd.DataFrame(reordered_rows)

    # ----------------------
    # 4. 원본 + (파라프+순서섞기)만 합치기
    # ----------------------
    reordered_df.to_csv(TRAIN_AUG_FILE, index=False) # TRAIN_AUG_FILE 사용
    print(f"완료! → {TRAIN_AUG_FILE}")

    # 모델 및 캐시 정리
    cleanup()
    print("데이터 증강 완료.")

# 3. 모델 훈련 (train.py 내용)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

def load_and_prepare_data():
    """데이터 로드 및 준비"""
    # 원본 및 증강 데이터 읽기
    df = pd.read_csv(TRAIN_FILE) # TRAIN_FILE 사용
    aug_df = pd.read_csv(TRAIN_AUG_FILE) # TRAIN_AUG_FILE 사용

    # 순열 tuple 컬럼 추가 (stratify split용)
    def tuple_from_row(row):
        return tuple(row[f"answer_{j}"] for j in range(4))
    df["permutation"] = df.apply(tuple_from_row, axis=1)

    # 원본 데이터 기준으로 split
    train_df, val_df = train_test_split(
        df,
        test_size=0.25,
        random_state=42,
        stratify=df["permutation"]
    )
    train_df = train_df.drop(columns=["permutation"])
    val_df = val_df.drop(columns=["permutation"])

    # train set에만 증강 데이터 추가
    full_train_df = pd.concat([train_df, aug_df], ignore_index=True)

    # 데이터 포맷팅
    def format_data(df):
        formatted = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            text = PROMPT_TEMPLATE.format( # PROMPT_TEMPLATE 사용
                sentence_0=row['sentence_0'],
                sentence_1=row['sentence_1'],
                sentence_2=row['sentence_2'],
                sentence_3=row['sentence_3'],
            ) + f" {row['answer_0']},{row['answer_1']},{row['answer_2']},{row['answer_3']}<|im_end|>"
            formatted.append({"text": text})
        return formatted

    train_data = format_data(full_train_df)
    val_data = format_data(val_df)

    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)

    print(f"훈련: {len(train_dataset)}, 검증: {len(val_dataset)}")
    return train_dataset, val_dataset


def setup_model():
    """모델 및 토크나이저 설정"""
    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    # 모델 로드
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )

    # LoRA 설정
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=LORA_TARGETS,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    return model, tokenizer

def train_model(train_dataset, val_dataset, model, tokenizer):
    """모델 훈련"""
    # 데이터 콜레이터
    response_template = "<|im_start|>assistant"
    collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

    # 훈련 인자
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUMULATION,
        learning_rate=LEARNING_RATE,
        max_steps=MAX_STEPS,
        warmup_steps=WARMUP_STEPS,
        save_steps=SAVE_STEPS,
        do_eval=True,
        eval_steps=SAVE_STEPS,
        logging_steps=50,
        fp16=True,
        bf16=False,
        optim="paged_adamw_8bit",
        gradient_checkpointing=False,
        group_by_length=True,
        report_to="none",
        save_total_limit=2,
        load_best_model_at_end=False,
    )

    # 트레이너
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        args=training_args,
        data_collator=collator,
    )

    # 훈련 시작
    print("훈련 시작...")
    trainer.train()

    # 모델 저장
    trainer.save_model()
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"모델 저장 완료: {OUTPUT_DIR}")

    return trainer

def run_training():
    print("훈련 파이프라인 시작...")
    print("데이터 준비...")
    train_dataset, val_dataset = load_and_prepare_data()

    print("모델 설정...")
    model, tokenizer = setup_model()

    print("훈련 시작...")
    trainer = train_model(train_dataset, val_dataset, model, tokenizer)

    print("훈련 완료!")
    # 훈련 후 모델과 토크나이저를 반환하여 추론 단계에서 재사용
    return model, tokenizer

# 4. 추론 (inference.py 내용)

def load_model_and_tokenizer_inference() -> Tuple[PeftModel, AutoTokenizer]:
    """모델과 토크나이저 로드 (로컬 어댑터 경로 사용)"""
    print("🔧 모델 로드 중...")

    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        cache_dir=CACHE_DIR
    )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=USE_4BIT,
        bnb_4bit_use_double_quant=BNB_4BIT_USE_DOUBLE_QUANT,
        bnb_4bit_quant_type=BNB_4BIT_QUANT_TYPE,
        bnb_4bit_compute_dtype=compute_dtype,
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
        cache_dir=CACHE_DIR
    )

    # === 로컬 어댑터 로드 ===

    model = PeftModel.from_pretrained(
        base_model,
        OUTPUT_DIR
    )
    model.eval()

    print("✅ 모델 로드 완료!")
    return model, tokenizer

def create_fewshot_prompt(sentences):
    """PROMPT_TEMPLATE을 사용하여 프롬프트 생성"""
    return PROMPT_TEMPLATE.format(
        sentence_0=sentences[0],
        sentence_1=sentences[1],
        sentence_2=sentences[2],
        sentence_3=sentences[3]
    )

def predict_batch_return_raw(sentences_batch: List[List[str]], model: PeftModel, tokenizer: AutoTokenizer) -> Tuple[List[str], List[str]]:
    """
    배치 예측 결과와 LLM 원본 답변(raw text) 반환.
    """
    prompts = [create_fewshot_prompt(sentences) for sentences in sentences_batch]
    messages_batch = [[{"role": "user", "content": prompt}] for prompt in prompts]
    texts = [tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
             for messages in messages_batch]
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            temperature=None,
            top_p=None,
            top_k=None,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.05
        )
    results = []
    raw_outputs = []
    for i, output in enumerate(outputs):
        input_length = inputs.input_ids[i].shape[0]
        generated = tokenizer.decode(output[input_length:], skip_special_tokens=True).strip()
        results.append(extract_order_enhanced(generated))
        raw_outputs.append(generated)
    return results, raw_outputs


def extract_order_enhanced(text: str) -> str:
    """강화된 파싱 함수: 모델 출력에서 문장 순서 추출"""

    # 1순위: 정확한 패턴들
    exact_patterns = [
        r'답[:：]\s*([0-3]),\s*([0-3]),\s*([0-3]),\s*([0-3])',
        r'답[:：]\s*([0-3])\s*,\s*([0-3])\\s*,\\s*([0-3])\\s*,\\s*([0-3])',
        r'순서[:：]\s*([0-3]),\\s*([0-3]),\\s*([0-3]),\\s*([0-3])',
        r'([0-3]),\\s*([0-3]),\\s*([0-3]),\\s*([0-3])',
        r'([0-3])\\s*→\\s*([0-3])\\s*→\\s*([0-3])\\s*→\\s*([0-3])',
        r'([0-3])\\s+([0-3])\\s+([0-3])\\s+([0-3])',
        r'([0-3])-([0-3])-([0-3])-([0-3])',
        r'([0-3])\\.([0-3])\\.([0-3])\\.([0-3])',
    ]

    for pattern in exact_patterns:
        match = re.search(pattern, text)
        if match:
            if len(match.groups()) == 4:
                result = [int(g) for g in match.groups()]
            else:
                result = [int(d) for d in match.group(1) if d.isdigit()]

            if len(result) == 4 and set(result) == {0,1,2,3}:
                return ''.join(map(str, result))

    # 2순위: 4자리 연속 숫자
    four_digit = re.search(r'\\b([0-3]{4})\\b', text)
    if four_digit:
        return four_digit.group(1)

    # 3순위: 순서/답 키워드 뒤 숫자들
    patterns = [r'순서[:：]\\s*([0-3\\s,]+)', r'답[:：]\\s*([0-3\\s,]+)', r'결과[:：]\\s*([0-3\\s,]+)']
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            numbers = re.findall(r'[0-3]', match.group(1))
            if len(numbers) >= 4:
                return ''.join(numbers[:4])

    # 4순위: 마지막 줄에서 숫자들
    lines = [line.strip() for line in text.split('\\n') if line.strip()]
    if lines:
        numbers = re.findall(r'[0-3]', lines[-1])
        if len(numbers) >= 4:
            return ''.join(numbers[:4])

    # 5순위: 전체에서 숫자들
    all_numbers = re.findall(r'[0-3]', text)
    if len(all_numbers) >= 4:
        return ''.join(all_numbers[-4:])

    # 파싱 실패시 원본 텍스트 반환
    return text

def extract_sentences(row: pd.Series) -> List[str]:
    """행에서 문장들 추출"""
    patterns = [
        ['sentence_0', 'sentence_1', 'sentence_2', 'sentence_3'],
        ['sent_0', 'sent_1', 'sent_2', 'sent_3'],
        ['text_0', 'text_1', 'text_2', 'text_3'],
        ['0', '1', '2', '3']
    ]

    for pattern in patterns:
        try:
            return [str(row[col]) for col in pattern]
        except KeyError:
            continue

    # 처음 4개 컬럼 사용
    return [str(row.iloc[i]) for i in range(min(4, len(row)))]

def process_result(predicted_order: str) -> Dict[str, Union[int, str]]:
    """예측 결과를 처리 (후처리 없음)"""
    if len(predicted_order) == 4 and predicted_order.isdigit():
        # 파싱 성공
        return {
            'answer_0': int(predicted_order[0]),
            'answer_1': int(predicted_order[1]),
            'answer_2': int(predicted_order[2]),
            'answer_3': int(predicted_order[3]),
            'raw_output': '',
            'parsing_status': 'SUCCESS'
        }
    else:
        # 파싱 실패
        return {
            'answer_0': '',
            'answer_1': '',
            'answer_2': '',
            'answer_3': '',
            'raw_output': predicted_order,
            'parsing_status': 'FAILED'
        }

def run_inference(input_file: str, output_file: str, model_inf=None, tokenizer_inf=None) -> pd.DataFrame:
    print("순수 AI 추론 시작...")
    start_time = time.time()

    # 모델/토크나이저 로드 (훈련에서 반환된 것이 없으면 새로 로드)
    if model_inf is None or tokenizer_inf is None:
        model, tokenizer = load_model_and_tokenizer_inference()
    else:
        model, tokenizer = model_inf, tokenizer_inf

    # 데이터 로드
    df = pd.read_csv(input_file)
    print(f"📂 데이터 로드: {len(df)}개 행")

    # 배치별 예측
    results = []
    total_batches = (len(df) + INFERENCE_BATCH_SIZE - 1) // INFERENCE_BATCH_SIZE

    print(f"🚀 배치 크기: {INFERENCE_BATCH_SIZE}, 총 배치: {total_batches}")

    for batch_idx in tqdm(range(total_batches), desc="배치 처리"):
        start_idx = batch_idx * INFERENCE_BATCH_SIZE
        end_idx = min(start_idx + INFERENCE_BATCH_SIZE, len(df))
        batch_rows = df.iloc[start_idx:end_idx]

        try:
            sentences_batch = [extract_sentences(row) for _, row in batch_rows.iterrows()]
            predicted_orders, raw_outputs = predict_batch_return_raw(sentences_batch, model, tokenizer)

            for i, (row_idx, _) in enumerate(batch_rows.iterrows()):
                result = process_result(predicted_orders[i])
                results.append({
                    'ID': f'TEST_{row_idx:04d}',
                    'answer_0': result['answer_0'],
                    'answer_1': result['answer_1'],
                    'answer_2': result['answer_2'],
                    'answer_3': result['answer_3'],
                    'context': raw_outputs[i],  # LLM 원본 답변
                    'raw_output': result['raw_output'],
                    'parsing_status': result['parsing_status'],
                    'sentence_0': sentences_batch[i][0],
                    'sentence_1': sentences_batch[i][1],
                    'sentence_2': sentences_batch[i][2],
                    'sentence_3': sentences_batch[i][3]
                })
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        except Exception as e:
            for i, (row_idx, _) in enumerate(batch_rows.iterrows()):
                results.append({
                    'ID': f'TEST_{row_idx:04d}',
                    'answer_0': '',
                    'answer_1': '',
                    'answer_2': '',
                    'answer_3': '',
                    'context': '',
                    'raw_output': f'BATCH_ERROR: {e}',
                    'parsing_status': 'ERROR',
                    'sentence_0': '',
                    'sentence_1': '',
                    'sentence_2': '',
                    'sentence_3': ''
                })

    results_df = pd.DataFrame(results)

    # (1) 결측치/파싱실패 행 재추론
    mask = results_df[['answer_0','answer_1','answer_2','answer_3']].isnull().any(axis=1) | (results_df['parsing_status'] != 'SUCCESS')
    failed_rows = results_df[mask]
    if len(failed_rows) > 0:
        print(f"⚠️ 결측/파싱실패 행 {len(failed_rows)}건, 재추론 시도")
        # 원본 문장들로 재추론
        sentences_batch = [
            [row['sentence_0'], row['sentence_1'], row['sentence_2'], row['sentence_3']]
            for _, row in failed_rows.iterrows()
        ]
        predicted_orders, raw_outputs = predict_batch_return_raw(sentences_batch, model, tokenizer)
        for idx, (row_idx, _) in enumerate(failed_rows.iterrows()):
            result = process_result(predicted_orders[idx])
            # 기존 결과 덮어쓰기
            results_df.loc[failed_rows.index[idx], 'answer_0'] = result['answer_0']
            results_df.loc[failed_rows.index[idx], 'answer_1'] = result['answer_1']
            results_df.loc[failed_rows.index[idx], 'answer_2'] = result['answer_2']
            results_df.loc[failed_rows.index[idx], 'answer_3'] = result['answer_3']
            results_df.loc[failed_rows.index[idx], 'context'] = raw_outputs[idx]
            results_df.loc[failed_rows.index[idx], 'raw_output'] = result['raw_output']
            results_df.loc[failed_rows.index[idx], 'parsing_status'] = result['parsing_status']

    # 결과 저장
    results_df.to_csv(output_file, index=False, encoding='utf-8-sig')

    # 성공률 등 통계
    success_count = (results_df['parsing_status'] == 'SUCCESS').sum()
    failed_count = (results_df['parsing_status'] == 'FAILED').sum()
    error_count = (results_df['parsing_status'] == 'ERROR').sum()
    total_count = len(results_df)
    success_rate = (success_count / total_count) * 100 if total_count > 0 else 0

    elapsed_time = time.time() - start_time
    samples_per_sec = len(df) / elapsed_time

    print(f"💾 순수 AI 추론 결과 저장: {output_file}")
    print(f"📊 파싱 성공률: {success_rate:.1f}% ({success_count}/{total_count})")
    print(f"   - 성공: {success_count}")
    print(f"   - 실패: {failed_count}")
    print(f"   - 에러: {error_count}")
    print(f"⏱️  처리 시간: {elapsed_time:.1f}초")
    print(f"🚀 처리 속도: {samples_per_sec:.1f} 샘플/초")

    return results_df



In [ ]:
# 1. 데이터 증강 실행
# Colab에서 처음 실행하거나 train_augmented_kor.csv를 새로 생성하려면 이 줄의 주석을 해제하세요.
# 데이터 증강은 시간이 오래 걸릴 수 있으며, GPU 메모리를 많이 사용합니다.
print("=====================================")
print("      단계 1: 데이터 증강 시작       ")
print("=====================================")
# run_augmentation()
print("데이터 증강 단계는 건너뛰었습니다. 기존 'train_augmented_kor.csv' 파일을 사용합니다.")

# 2. 모델 훈련 실행
# 모델을 처음 훈련시키거나 재훈련시키려면 이 줄의 주석을 해제하세요.
# 훈련된 모델은 OUTPUT_DIR에 저장됩니다.
print("\n=====================================")
print("       단계 2: 모델 훈련 시작        ")
print("=====================================")
# trained_model, trained_tokenizer = run_training()
trained_model, trained_tokenizer = None, None # 훈련 건너뛰고 추론 시 모델을 새로 로드하도록 설정
print("모델 훈련 단계는 건너뛰었습니다. 추론 시 모델을 새로 로드합니다.")

# 3. 추론 실행
# 훈련된 모델로 test.csv에 대한 예측을 수행하고 predictions.csv를 생성합니다.
print("\n=====================================")
print("        단계 3: 추론 시작          ")
print("=====================================")
# 훈련된 모델과 토크나이저를 추론 함수에 전달 (훈련을 건너뛰었다면 None이 전달되어 새로 로드됩니다)
final_predictions_df = run_inference(TEST_FILE, PREDICTIONS_FILE, trained_model, trained_tokenizer)

print("\n=====================================")
print("         모든 프로세스 완료!         ")
print("=====================================")
print(f"최종 예측 결과 파일: {PREDICTIONS_FILE}")
print("수고하셨습니다!")

      단계 1: 데이터 증강 시작       
데이터 증강 단계는 건너뛰었습니다. 기존 'train_augmented_kor.csv' 파일을 사용합니다.

       단계 2: 모델 훈련 시작        
모델 훈련 단계는 건너뛰었습니다. 추론 시 모델을 새로 로드합니다.

        단계 3: 추론 시작          
순수 AI 추론 시작...
🔧 모델 로드 중...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

KeyboardInterrupt: 